## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import weather_api_key
import config

# Configure gmaps API key
#gmaps.configure(api_key=weather_api_key)
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.tail()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3419,3419,Nerchinskiy Zavod,RU,51.3088,119.6100,-2.20,100,100,2.59,overcast clouds
3420,3420,Sumbe,AO,-11.2061,13.8437,76.91,91,88,4.52,moderate rain
3421,3421,Opuwo,NaN,-18.0607,13.8400,73.13,63,37,11.05,light rain
3422,3422,Nisia Floresta,BR,-6.0911,-35.2086,83.21,78,11,10.31,few clouds
3423,3423,El Reno,US,35.5323,-97.9550,58.21,21,1,24.16,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is your desired minimum temperature for your trip? "))
max_temp = float(input("What is your desired maximum temperature for your trip? ")) 

What is your desired minimum temperature for your trip? 75
What is your desired maximum temperature for your trip? 90


In [ ]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

In [ ]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

In [6]:
len(preferred_cities_df)

724

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
686,686,Puerto Ayora,EC,-0.7393,-90.3518,75.16,92,99,7.74,overcast clouds
691,691,Rikitea,PF,-23.1203,-134.9692,77.65,76,87,8.63,light rain
694,694,Patnanungan,PH,14.7833,122.1833,78.67,83,100,24.14,overcast clouds
700,700,Georgetown,MY,5.4112,100.3354,79.90,82,20,0.00,few clouds
701,701,Hilo,US,19.7297,-155.0900,79.11,59,75,5.75,broken clouds
...,...,...,...,...,...,...,...,...,...,...
3414,3414,Daru,PG,-9.0763,143.2092,79.16,87,100,12.44,overcast clouds
3415,3415,Roebourne,AU,-20.7833,117.1333,89.94,22,0,10.56,light rain
3417,3417,San Andres,CO,12.5847,-81.7006,82.42,78,75,8.05,broken clouds
3420,3420,Sumbe,AO,-11.2061,13.8437,76.91,91,88,4.52,moderate rain


In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
686,Puerto Ayora,EC,75.16,overcast clouds,-0.7393,-90.3518,
691,Rikitea,PF,77.65,light rain,-23.1203,-134.9692,
694,Patnanungan,PH,78.67,overcast clouds,14.7833,122.1833,
700,Georgetown,MY,79.90,few clouds,5.4112,100.3354,
701,Hilo,US,79.11,broken clouds,19.7297,-155.0900,
704,Arraial Do Cabo,BR,76.69,light rain,-22.9661,-42.0278,
715,Mahebourg,MU,75.49,light intensity shower rain,-20.4081,57.7000,
717,Vaini,TO,82.56,light rain,-21.2000,-175.2000,
719,Adiake,CI,83.79,scattered clouds,5.2863,-3.3040,
725,Gonaives,HT,82.38,clear sky,19.4500,-72.6833,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']

    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError): 
        print("No hotel found...skipping.")

No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.


ConnectionError: ('Connection aborted.', TimeoutError(10060, 'A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond', None, 10060, None))

In [10]:
hotels = requests.get(base_url, params=params).json()
hotels  

{'html_attributions': [],
 'next_page_token': 'Aap_uECu56AyWKNf92HZ42tSaSdWTknqZ5sDwxev7BrHvZ6FEBZ7iT7aULFdgaw15DpZOeFkHOjnBsDX6hTL1Bm5e5Cphg-MSj9Qg4NcbiDkjjKvDZjSwLFNPZDDS93gf3BYn2gq-kdf2G8tQZMy-C_14Pch36TKtbYbpi8hSk4kMzXq8UkKbuKf6lqC1Q98KfBBH-0-l2PSuLS0HZOrMP17WJ8IVyBniSErQE8uFc_s0oLHXwihFe4NcGLSN5NXtaUJ-YnoO-4nSHAdBiFOpj_kpYzJLnGg3A8T24q95VuEWzgu6CON-YU3Qx23sV4f2-yRRkG3M8AT4J4sS4QvIxTl1g_WWvbBGMUlu0DsjLfVxAE70CimRk5ayeStkB_dwkJeJhUg4i-rXWaWxbSAxH8i7lpFfvI-YFP1OuUyMxx2dOVyjmgRJmSFJs_D',
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': 20.7555111, 'lng': -156.4575206},
    'viewport': {'northeast': {'lat': 20.7565973302915, 'lng': -156.45613945},
     'southwest': {'lat': 20.7538993697085, 'lng': -156.45892965}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
   'icon_background_color': '#909CE1',
   'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet',
   'name': 'Maui S

In [11]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = (hotel_df.replace(r'^\s*$', np.nan, regex=True)).dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
686,Puerto Ayora,EC,75.16,overcast clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
691,Rikitea,PF,77.65,light rain,-23.1203,-134.9692,People ThankYou
694,Patnanungan,PH,78.67,overcast clouds,14.7833,122.1833,Villa Caesar
700,Georgetown,MY,79.90,few clouds,5.4112,100.3354,Cititel Penang
701,Hilo,US,79.11,broken clouds,19.7297,-155.0900,Hilo Hawaiian Hotel


In [12]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]
locations

,Lat,Lng
686,-0.7393,-90.3518
691,-23.1203,-134.9692
694,14.7833,122.1833
700,5.4112,100.3354
701,19.7297,-155.0900
...,...,...
1684,-38.2667,141.9667
1687,-17.7333,-149.1500
1693,7.5836,-75.4007
1694,9.4942,50.8122


In [14]:
# 11a. Add a marker layer for each city to the map. 
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(70.0, 71.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))